In [1]:
from embedder import Embedder
from data_provider import DataProvider

In [2]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mounts/data/proj/faeze/.cache/hf/'
os.environ['HF_HOME'] = '/mounts/data/proj/faeze/.cache/hf/'
os.environ['HF_DATASETS_CACHE'] = '/mounts/data/proj/faeze/.cache/hf/'
os.environ['TORCH_HUB'] = '/mounts/data/proj/faeze/.cache/torch/'
os.environ['TORCH_HOME'] = '/mounts/data/proj/faeze/.cache/torch/'
os.environ["WANDB_DIR"] = '/mounts/data/proj/faeze/.cache/wandb/'

In [3]:
data_provider_ = DataProvider()

In [4]:
datas = ["bas19_es", "for19_pt", "has21_hi", "ous19_ar", "ous19_fr", "san20_it", "gahd24_de", "xdomain_tr"]
languages = [i.split('_')[-1] for i in datas]

In [5]:
data_names = [i+'-2000-rs5' for i in datas] #${dataset}-${split}-${RSS[i]}

In [6]:
datasets = data_provider_.load_datasets(data_names, languages)

Using the latest cached version of the module from /mounts/data/corp/huggingface/modules/datasets_modules/datasets/baseline_data/b7251845183b39abbb6d9035ffe620e6b81f4de92ea1733dfa4fa18d060d6612 (last modified on Wed Mar 12 17:18:48 2025) since it couldn't be found locally at baseline_data, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /mounts/data/corp/huggingface/modules/datasets_modules/datasets/baseline_data/b7251845183b39abbb6d9035ffe620e6b81f4de92ea1733dfa4fa18d060d6612 (last modified on Wed Mar 12 17:18:48 2025) since it couldn't be found locally at baseline_data, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /mounts/data/corp/huggingface/modules/datasets_modules/datasets/baseline_data/b7251845183b39abbb6d9035ffe620e6b81f4de92ea1733dfa4fa18d060d6612 (last modified on Wed Mar 12 17:18:48 2025) since it couldn't be found locally at baseline_data, or remotely on the Hugging Face Hub.
Using the latest

In [7]:
datasets

[{'name': 'bas19_es-2000-rs5',
  'data': DatasetDict({
      train: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
      validation: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 500
      })
      test: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
      hate_check: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 3745
      })
      hate_day: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2058
      })
  }),
  'language': 'es'},
 {'name': 'for19_pt-2000-rs5',
  'data': DatasetDict({
      train: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
      validation: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 500
      })
      test: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
      hate_check: Dataset({
     

In [8]:

embedder_ = Embedder(model_name="m3")
embeddings, metadatas = embedder_.embed_datasets(datasets, ['train'], False)

Embedding sentences: 100%|██████████| 8/8 [00:08<00:00,  1.04s/it]


In [9]:
import numpy as np
embed = np.vstack(embeddings)
metadata = metadatas

In [10]:
len(metadata), embed.shape

(16000, (16000, 1024))

In [11]:
embed.shape

(16000, 1024)

In [12]:
# clusters = Embedder.cluster_embeddings(embed, metadata, 150)
# clusters

In [13]:
embed.shape, len(metadata)

((16000, 1024), 16000)

In [14]:
# Embedder.visualize_embeddings(embed, metadatas, reduction_method="tsne")

In [15]:
import pandas as pd
import numpy as np
metadata_df = pd.DataFrame(metadatas)

In [16]:
metadata_df['dataset_name'].value_counts()

dataset_name
bas19_es-2000-rs5      2000
for19_pt-2000-rs5      2000
has21_hi-2000-rs5      2000
ous19_ar-2000-rs5      2000
ous19_fr-2000-rs5      2000
san20_it-2000-rs5      2000
gahd24_de-2000-rs5     2000
xdomain_tr-2000-rs5    2000
Name: count, dtype: int64

In [17]:
from retriever import Retriever
retriever = Retriever(embedder_.embedding_dim, index_type="HNSW")

In [18]:
retriever.device

'cuda'

In [19]:
retriever.load_index("/mounts/data/proj/faeze/data_efficient_hate/models/retriever/all_multilingual_with_m3/")

In [20]:
# retrieved_samples = retriever.retrieve()

In [21]:
# retriever.save_index("/mounts/data/proj/faeze/data_efficient_hate/models/retriever/temp/")

In [22]:
datas

['bas19_es',
 'for19_pt',
 'has21_hi',
 'ous19_ar',
 'ous19_fr',
 'san20_it',
 'gahd24_de',
 'xdomain_tr']

In [150]:
from collections import Counter
target_to_source = dict()
target_to_language = dict()
target_to_label = dict()

for i, data in enumerate(datas):
    embedd = embeddings[i]
    excludeds = [data]
    if 'gahd' in data:
        excludeds.append('dyn21_en')
    elif 'xdomain' in data:
        excludeds.append('xdomain_en')

    retrieved = retriever.retrieve_multiple_queries(embedd, num_retrieved=2000, exclude_datasets=excludeds)

    # Extract relevant metadata
    dataset_names = [item['metadata']['dataset_name'] for item in retrieved]
    languages = [item['metadata']['language'] for item in retrieved]
    labels = [item['metadata']['label'] for item in retrieved]

    # Count occurrences
    dataset_counter = Counter(dataset_names)
    language_counter = Counter(languages)
    label_counter = Counter(labels)

    total_retrieved = len(retrieved)
    print(data, dataset_counter.keys(), dataset_counter.values())
    target_to_source[data] = dict(dataset_counter)
    target_to_language[data] = dict(language_counter)
    target_to_label[data] = dict(label_counter)

    # Find most common and calculate percentages
    # most_common_dataset, dataset_count = dataset_counter.most_common(1)[0]
    # most_common_language, language_count = language_counter.most_common(1)[0]
    # most_common_label, label_count = label_counter.most_common(1)[0]
    #
    #
    # dataset_percentage = (dataset_count / total_retrieved) * 100
    # language_percentage = (language_count / total_retrieved) * 100
    # label_percentage = (label_count / total_retrieved) * 100
    #
    # # Print results
    # print(data)
    # print(f"Most retrieved dataset_name: {most_common_dataset} ({dataset_percentage:.2f}%)")
    # print(f"Most retrieved language: {most_common_language} ({language_percentage:.2f}%)")
    # print(f"Most retrieved label: {most_common_label} ({label_percentage:.2f}%)")
    # print()


bas19_es dict_keys(['ken20_en', 'fou18_en', 'ous19_fr', 'has21_hi', 'for19_pt', 'gahd24_de', 'xdomain_en', 'san20_it', 'ous19_ar', 'xplain_en', 'dyn21_en', 'implicit_en', 'xdomain_tr']) dict_values([315, 304, 129, 49, 191, 31, 135, 233, 84, 46, 279, 45, 159])
for19_pt dict_keys(['ken20_en', 'has21_hi', 'fou18_en', 'bas19_es', 'ous19_ar', 'ous19_fr', 'xdomain_tr', 'dyn21_en', 'gahd24_de', 'xdomain_en', 'san20_it', 'implicit_en', 'xplain_en']) dict_values([266, 55, 477, 195, 89, 152, 135, 176, 34, 206, 102, 84, 29])
has21_hi dict_keys(['ken20_en', 'fou18_en', 'for19_pt', 'bas19_es', 'ous19_ar', 'ous19_fr', 'xdomain_tr', 'dyn21_en', 'san20_it', 'implicit_en', 'xplain_en', 'xdomain_en', 'gahd24_de']) dict_values([275, 308, 94, 309, 287, 155, 224, 69, 107, 10, 22, 138, 2])
ous19_ar dict_keys(['ous19_fr', 'ken20_en', 'for19_pt', 'has21_hi', 'bas19_es', 'fou18_en', 'gahd24_de', 'xdomain_tr', 'dyn21_en', 'xdomain_en', 'san20_it', 'implicit_en', 'xplain_en']) dict_values([260, 268, 139, 81, 170

In [151]:
target_to_source

{'bas19_es': {'ken20_en': 315,
  'fou18_en': 304,
  'ous19_fr': 129,
  'has21_hi': 49,
  'for19_pt': 191,
  'gahd24_de': 31,
  'xdomain_en': 135,
  'san20_it': 233,
  'ous19_ar': 84,
  'xplain_en': 46,
  'dyn21_en': 279,
  'implicit_en': 45,
  'xdomain_tr': 159},
 'for19_pt': {'ken20_en': 266,
  'has21_hi': 55,
  'fou18_en': 477,
  'bas19_es': 195,
  'ous19_ar': 89,
  'ous19_fr': 152,
  'xdomain_tr': 135,
  'dyn21_en': 176,
  'gahd24_de': 34,
  'xdomain_en': 206,
  'san20_it': 102,
  'implicit_en': 84,
  'xplain_en': 29},
 'has21_hi': {'ken20_en': 275,
  'fou18_en': 308,
  'for19_pt': 94,
  'bas19_es': 309,
  'ous19_ar': 287,
  'ous19_fr': 155,
  'xdomain_tr': 224,
  'dyn21_en': 69,
  'san20_it': 107,
  'implicit_en': 10,
  'xplain_en': 22,
  'xdomain_en': 138,
  'gahd24_de': 2},
 'ous19_ar': {'ous19_fr': 260,
  'ken20_en': 268,
  'for19_pt': 139,
  'has21_hi': 81,
  'bas19_es': 170,
  'fou18_en': 269,
  'gahd24_de': 40,
  'xdomain_tr': 231,
  'dyn21_en': 221,
  'xdomain_en': 151,
  's

In [159]:
source_tasks = datas + ["dyn21_en", "fou18_en", "ken20_en", "xplain_en", "implicit_en", "xdomain_en"]
target_tasks = datas

In [ ]:
rs1_source = {'bas19_es': {'fou18_en': 325,
  'for19_pt': 195,
  'ken20_en': 309,
  'ous19_fr': 125,
  'has21_hi': 42,
  'ous19_ar': 98,
  'dyn21_en': 292,
  'san20_it': 222,
  'xdomain_tr': 132,
  'xdomain_en': 134,
  'xplain_en': 50,
  'implicit_en': 52,
  'gahd24_de': 24},
 'for19_pt': {'fou18_en': 476,
  'ken20_en': 255,
  'bas19_es': 200,
  'ous19_ar': 74,
  'san20_it': 118,
  'ous19_fr': 180,
  'dyn21_en': 197,
  'has21_hi': 47,
  'gahd24_de': 25,
  'xdomain_en': 195,
  'implicit_en': 84,
  'xplain_en': 39,
  'xdomain_tr': 110},
 'has21_hi': {'ken20_en': 267,
  'fou18_en': 322,
  'for19_pt': 91,
  'bas19_es': 298,
  'ous19_ar': 281,
  'ous19_fr': 145,
  'dyn21_en': 73,
  'xdomain_en': 151,
  'san20_it': 106,
  'xdomain_tr': 235,
  'implicit_en': 11,
  'xplain_en': 17,
  'gahd24_de': 3},
 'ous19_ar': {'ken20_en': 281,
  'fou18_en': 259,
  'bas19_es': 163,
  'ous19_fr': 228,
  'has21_hi': 81,
  'for19_pt': 132,
  'xdomain_en': 141,
  'san20_it': 96,
  'dyn21_en': 241,
  'xdomain_tr': 254,
  'gahd24_de': 38,
  'implicit_en': 54,
  'xplain_en': 32},
 'ous19_fr': {'bas19_es': 215,
  'has21_hi': 70,
  'gahd24_de': 29,
  'for19_pt': 232,
  'ous19_ar': 170,
  'ken20_en': 246,
  'fou18_en': 434,
  'san20_it': 227,
  'xdomain_tr': 62,
  'xdomain_en': 108,
  'dyn21_en': 103,
  'xplain_en': 50,
  'implicit_en': 54},
 'san20_it': {'dyn21_en': 327,
  'gahd24_de': 43,
  'xplain_en': 77,
  'ken20_en': 398,
  'ous19_ar': 42,
  'xdomain_en': 165,
  'ous19_fr': 161,
  'for19_pt': 119,
  'has21_hi': 31,
  'bas19_es': 120,
  'fou18_en': 140,
  'implicit_en': 111,
  'xdomain_tr': 266},
 'gahd24_de': {'ken20_en': 612,
  'implicit_en': 465,
  'fou18_en': 108,
  'xdomain_en': 270,
  'bas19_es': 30,
  'for19_pt': 37,
  'ous19_fr': 36,
  'xdomain_tr': 177,
  'xplain_en': 200,
  'san20_it': 50,
  'ous19_ar': 10,
  'has21_hi': 5},
 'xdomain_tr': {'ken20_en': 386,
  'bas19_es': 90,
  'for19_pt': 121,
  'dyn21_en': 517,
  'ous19_ar': 59,
  'has21_hi': 39,
  'ous19_fr': 77,
  'fou18_en': 288,
  'implicit_en': 81,
  'san20_it': 170,
  'xplain_en': 103,
  'gahd24_de': 69}}

rs2_source = {'bas19_es': {'dyn21_en': 250,
  'ous19_fr': 133,
  'xdomain_en': 126,
  'ken20_en': 322,
  'has21_hi': 40,
  'fou18_en': 308,
  'for19_pt': 198,
  'xplain_en': 61,
  'san20_it': 253,
  'gahd24_de': 32,
  'implicit_en': 47,
  'ous19_ar': 87,
  'xdomain_tr': 143},
 'for19_pt': {'fou18_en': 494,
  'ken20_en': 264,
  'ous19_fr': 178,
  'xdomain_en': 191,
  'ous19_ar': 73,
  'dyn21_en': 182,
  'bas19_es': 206,
  'has21_hi': 58,
  'san20_it': 89,
  'gahd24_de': 26,
  'implicit_en': 71,
  'xdomain_tr': 134,
  'xplain_en': 34},
 'has21_hi': {'ken20_en': 271,
  'for19_pt': 87,
  'bas19_es': 306,
  'ous19_fr': 160,
  'ous19_ar': 302,
  'fou18_en': 325,
  'xdomain_tr': 213,
  'dyn21_en': 63,
  'implicit_en': 9,
  'san20_it': 109,
  'xdomain_en': 138,
  'xplain_en': 15,
  'gahd24_de': 2},
 'ous19_ar': {'ken20_en': 281,
  'has21_hi': 82,
  'ous19_fr': 234,
  'for19_pt': 130,
  'bas19_es': 163,
  'fou18_en': 253,
  'xdomain_en': 137,
  'gahd24_de': 33,
  'san20_it': 102,
  'xdomain_tr': 273,
  'dyn21_en': 238,
  'implicit_en': 44,
  'xplain_en': 30},
 'ous19_fr': {'fou18_en': 442,
  'gahd24_de': 24,
  'bas19_es': 204,
  'ous19_ar': 169,
  'ken20_en': 247,
  'for19_pt': 244,
  'xdomain_tr': 81,
  'has21_hi': 60,
  'xplain_en': 45,
  'san20_it': 208,
  'dyn21_en': 112,
  'xdomain_en': 106,
  'implicit_en': 58},
 'san20_it': {'for19_pt': 132,
  'dyn21_en': 311,
  'ken20_en': 326,
  'bas19_es': 146,
  'ous19_fr': 159,
  'fou18_en': 150,
  'xdomain_en': 160,
  'ous19_ar': 41,
  'xdomain_tr': 279,
  'has21_hi': 24,
  'gahd24_de': 49,
  'xplain_en': 107,
  'implicit_en': 116},
 'gahd24_de': {'implicit_en': 459,
  'for19_pt': 27,
  'ken20_en': 621,
  'xplain_en': 202,
  'ous19_fr': 22,
  'bas19_es': 30,
  'fou18_en': 120,
  'xdomain_en': 309,
  'xdomain_tr': 140,
  'san20_it': 61,
  'ous19_ar': 7,
  'has21_hi': 2},
 'xdomain_tr': {'fou18_en': 287,
  'ken20_en': 362,
  'bas19_es': 78,
  'san20_it': 205,
  'dyn21_en': 514,
  'ous19_fr': 50,
  'for19_pt': 126,
  'xplain_en': 105,
  'has21_hi': 65,
  'ous19_ar': 44,
  'gahd24_de': 67,
  'implicit_en': 97}}
rs3_source = {'bas19_es': {'dyn21_en': 272,
  'ous19_ar': 82,
  'ken20_en': 332,
  'ous19_fr': 137,
  'has21_hi': 41,
  'fou18_en': 317,
  'for19_pt': 181,
  'xplain_en': 60,
  'gahd24_de': 20,
  'xdomain_en': 142,
  'san20_it': 229,
  'xdomain_tr': 145,
  'implicit_en': 42},
 'for19_pt': {'fou18_en': 485,
  'ken20_en': 257,
  'bas19_es': 224,
  'san20_it': 107,
  'ous19_fr': 155,
  'dyn21_en': 189,
  'xdomain_en': 197,
  'ous19_ar': 82,
  'has21_hi': 55,
  'xdomain_tr': 93,
  'implicit_en': 82,
  'xplain_en': 44,
  'gahd24_de': 30},
 'has21_hi': {'ken20_en': 273,
  'fou18_en': 323,
  'for19_pt': 96,
  'bas19_es': 316,
  'ous19_fr': 158,
  'ous19_ar': 295,
  'xdomain_tr': 210,
  'dyn21_en': 56,
  'san20_it': 105,
  'implicit_en': 10,
  'xplain_en': 21,
  'xdomain_en': 134,
  'gahd24_de': 3},
 'ous19_ar': {'fou18_en': 258,
  'bas19_es': 154,
  'ken20_en': 288,
  'ous19_fr': 246,
  'for19_pt': 137,
  'has21_hi': 78,
  'san20_it': 87,
  'xdomain_en': 139,
  'gahd24_de': 34,
  'dyn21_en': 266,
  'xdomain_tr': 238,
  'implicit_en': 46,
  'xplain_en': 29},
 'ous19_fr': {'bas19_es': 202,
  'for19_pt': 251,
  'fou18_en': 438,
  'has21_hi': 60,
  'gahd24_de': 25,
  'ous19_ar': 147,
  'ken20_en': 265,
  'san20_it': 233,
  'dyn21_en': 93,
  'xdomain_tr': 71,
  'xdomain_en': 114,
  'xplain_en': 46,
  'implicit_en': 55},
 'san20_it': {'dyn21_en': 358,
  'for19_pt': 112,
  'ken20_en': 373,
  'gahd24_de': 51,
  'ous19_ar': 34,
  'xdomain_tr': 272,
  'ous19_fr': 145,
  'xdomain_en': 151,
  'bas19_es': 144,
  'xplain_en': 74,
  'fou18_en': 127,
  'implicit_en': 142,
  'has21_hi': 17},
 'gahd24_de': {'ken20_en': 617,
  'implicit_en': 432,
  'fou18_en': 131,
  'xplain_en': 196,
  'xdomain_en': 321,
  'for19_pt': 37,
  'ous19_fr': 31,
  'bas19_es': 28,
  'san20_it': 57,
  'xdomain_tr': 142,
  'ous19_ar': 6,
  'has21_hi': 2},
 'xdomain_tr': {'ous19_fr': 77,
  'for19_pt': 111,
  'ken20_en': 405,
  'fou18_en': 308,
  'has21_hi': 60,
  'dyn21_en': 483,
  'gahd24_de': 69,
  'ous19_ar': 58,
  'san20_it': 173,
  'bas19_es': 73,
  'implicit_en': 95,
  'xplain_en': 88}}

rs4_source = {'bas19_es': {'for19_pt': 167,
  'ous19_fr': 120,
  'ken20_en': 337,
  'fou18_en': 342,
  'has21_hi': 48,
  'dyn21_en': 297,
  'xdomain_en': 118,
  'san20_it': 224,
  'ous19_ar': 87,
  'xdomain_tr': 141,
  'xplain_en': 51,
  'implicit_en': 44,
  'gahd24_de': 24},
 'for19_pt': {'ken20_en': 251,
  'has21_hi': 49,
  'fou18_en': 510,
  'bas19_es': 212,
  'san20_it': 118,
  'ous19_fr': 178,
  'xdomain_tr': 116,
  'ous19_ar': 94,
  'xdomain_en': 182,
  'dyn21_en': 167,
  'gahd24_de': 39,
  'implicit_en': 59,
  'xplain_en': 25},
 'has21_hi': {'ken20_en': 277,
  'fou18_en': 314,
  'bas19_es': 313,
  'ous19_ar': 289,
  'ous19_fr': 160,
  'for19_pt': 89,
  'xdomain_tr': 214,
  'dyn21_en': 68,
  'san20_it': 107,
  'implicit_en': 11,
  'xplain_en': 21,
  'xdomain_en': 134,
  'gahd24_de': 3},
 'ous19_ar': {'ken20_en': 270,
  'ous19_fr': 234,
  'has21_hi': 80,
  'for19_pt': 136,
  'bas19_es': 152,
  'fou18_en': 240,
  'xdomain_en': 145,
  'gahd24_de': 33,
  'san20_it': 108,
  'dyn21_en': 274,
  'xdomain_tr': 248,
  'implicit_en': 51,
  'xplain_en': 29},
 'ous19_fr': {'fou18_en': 446,
  'bas19_es': 218,
  'ous19_ar': 149,
  'for19_pt': 242,
  'ken20_en': 233,
  'xdomain_tr': 79,
  'has21_hi': 64,
  'san20_it': 228,
  'gahd24_de': 23,
  'dyn21_en': 115,
  'xdomain_en': 113,
  'xplain_en': 44,
  'implicit_en': 46},
 'san20_it': {'dyn21_en': 355,
  'ken20_en': 369,
  'ous19_fr': 142,
  'xdomain_tr': 266,
  'for19_pt': 124,
  'fou18_en': 154,
  'bas19_es': 135,
  'gahd24_de': 43,
  'has21_hi': 32,
  'xdomain_en': 134,
  'xplain_en': 76,
  'implicit_en': 122,
  'ous19_ar': 48},
 'gahd24_de': {'ken20_en': 604,
  'ous19_fr': 45,
  'implicit_en': 436,
  'xplain_en': 199,
  'xdomain_en': 273,
  'for19_pt': 44,
  'san20_it': 62,
  'fou18_en': 116,
  'xdomain_tr': 174,
  'has21_hi': 6,
  'bas19_es': 27,
  'ous19_ar': 14},
 'xdomain_tr': {'ken20_en': 403,
  'dyn21_en': 551,
  'for19_pt': 79,
  'bas19_es': 64,
  'fou18_en': 262,
  'ous19_fr': 60,
  'ous19_ar': 45,
  'san20_it': 213,
  'has21_hi': 43,
  'xplain_en': 117,
  'gahd24_de': 81,
  'implicit_en': 82}}

rs5_source = {'bas19_es': {'ken20_en': 315,
  'fou18_en': 304,
  'ous19_fr': 129,
  'has21_hi': 49,
  'for19_pt': 191,
  'gahd24_de': 31,
  'xdomain_en': 135,
  'san20_it': 233,
  'ous19_ar': 84,
  'xplain_en': 46,
  'dyn21_en': 279,
  'implicit_en': 45,
  'xdomain_tr': 159},
 'for19_pt': {'ken20_en': 266,
  'has21_hi': 55,
  'fou18_en': 477,
  'bas19_es': 195,
  'ous19_ar': 89,
  'ous19_fr': 152,
  'xdomain_tr': 135,
  'dyn21_en': 176,
  'gahd24_de': 34,
  'xdomain_en': 206,
  'san20_it': 102,
  'implicit_en': 84,
  'xplain_en': 29},
 'has21_hi': {'ken20_en': 275,
  'fou18_en': 308,
  'for19_pt': 94,
  'bas19_es': 309,
  'ous19_ar': 287,
  'ous19_fr': 155,
  'xdomain_tr': 224,
  'dyn21_en': 69,
  'san20_it': 107,
  'implicit_en': 10,
  'xplain_en': 22,
  'xdomain_en': 138,
  'gahd24_de': 2},
 'ous19_ar': {'ous19_fr': 260,
  'ken20_en': 268,
  'for19_pt': 139,
  'has21_hi': 81,
  'bas19_es': 170,
  'fou18_en': 269,
  'gahd24_de': 40,
  'xdomain_tr': 231,
  'dyn21_en': 221,
  'xdomain_en': 151,
  'san20_it': 99,
  'implicit_en': 46,
  'xplain_en': 25},
 'ous19_fr': {'for19_pt': 229,
  'gahd24_de': 32,
  'bas19_es': 219,
  'ous19_ar': 174,
  'ken20_en': 254,
  'xdomain_tr': 76,
  'has21_hi': 58,
  'fou18_en': 431,
  'san20_it': 243,
  'dyn21_en': 98,
  'xdomain_en': 108,
  'xplain_en': 35,
  'implicit_en': 43},
 'san20_it': {'for19_pt': 111,
  'xplain_en': 75,
  'gahd24_de': 50,
  'ken20_en': 361,
  'bas19_es': 145,
  'fou18_en': 122,
  'ous19_fr': 151,
  'xdomain_tr': 298,
  'dyn21_en': 341,
  'xdomain_en': 176,
  'implicit_en': 106,
  'has21_hi': 24,
  'ous19_ar': 40},
 'gahd24_de': {'implicit_en': 451,
  'xdomain_en': 275,
  'ken20_en': 582,
  'xplain_en': 209,
  'ous19_fr': 34,
  'bas19_es': 25,
  'fou18_en': 124,
  'for19_pt': 47,
  'xdomain_tr': 177,
  'san20_it': 67,
  'ous19_ar': 8,
  'has21_hi': 1},
 'xdomain_tr': {'fou18_en': 299,
  'dyn21_en': 511,
  'for19_pt': 109,
  'ous19_fr': 52,
  'has21_hi': 58,
  'implicit_en': 94,
  'ken20_en': 351,
  'xplain_en': 104,
  'ous19_ar': 40,
  'bas19_es': 97,
  'san20_it': 204,
  'gahd24_de': 81}}

In [166]:
from collections import defaultdict
rs_sources = [rs1_source, rs2_source, rs3_source, rs4_source, rs5_source]


# Store both total and count for averaging
value_sum = defaultdict(lambda: defaultdict(int))
value_count = defaultdict(lambda: defaultdict(int))

# Accumulate totals and counts
for rs in rs_sources:
    for target, sources in rs.items():
        for source, value in sources.items():
            value_sum[target][source] += value/2000
            value_count[target][source] += 1

# Compute averages
avg_dict = {
    target: {
        source: round(value_sum[target][source] * 100 / value_count[target][source], 2)
        for source in sources
    }
    for target, sources in value_sum.items()
}

# Print a sample
for target, sources in list(avg_dict.items()):
    print(f"{target}: {sources}")


bas19_es: {'fou18_en': 15.96, 'for19_pt': 9.32, 'ken20_en': 16.15, 'ous19_fr': 6.44, 'has21_hi': 2.2, 'ous19_ar': 4.38, 'dyn21_en': 13.9, 'san20_it': 11.61, 'xdomain_tr': 7.2, 'xdomain_en': 6.55, 'xplain_en': 2.68, 'implicit_en': 2.3, 'gahd24_de': 1.31}
for19_pt: {'fou18_en': 24.42, 'ken20_en': 12.93, 'bas19_es': 10.37, 'ous19_ar': 4.12, 'san20_it': 5.34, 'ous19_fr': 8.43, 'dyn21_en': 9.11, 'has21_hi': 2.64, 'gahd24_de': 1.54, 'xdomain_en': 9.71, 'implicit_en': 3.8, 'xplain_en': 1.71, 'xdomain_tr': 5.88}
has21_hi: {'ken20_en': 13.63, 'fou18_en': 15.92, 'for19_pt': 4.57, 'bas19_es': 15.42, 'ous19_ar': 14.54, 'ous19_fr': 7.78, 'dyn21_en': 3.29, 'xdomain_en': 6.95, 'san20_it': 5.34, 'xdomain_tr': 10.96, 'implicit_en': 0.51, 'xplain_en': 0.96, 'gahd24_de': 0.13}
ous19_ar: {'ken20_en': 13.88, 'fou18_en': 12.79, 'bas19_es': 8.02, 'ous19_fr': 12.02, 'has21_hi': 4.02, 'for19_pt': 6.74, 'xdomain_en': 7.13, 'san20_it': 4.92, 'dyn21_en': 12.4, 'xdomain_tr': 12.44, 'gahd24_de': 1.78, 'implicit_en'

In [153]:
target_to_language

{'bas19_es': {'en': 1124,
  'fr': 129,
  'hi': 49,
  'pt': 191,
  'de': 31,
  'it': 233,
  'ar': 84,
  'tr': 159},
 'for19_pt': {'en': 1238,
  'hi': 55,
  'es': 195,
  'ar': 89,
  'fr': 152,
  'tr': 135,
  'de': 34,
  'it': 102},
 'has21_hi': {'en': 822,
  'pt': 94,
  'es': 309,
  'ar': 287,
  'fr': 155,
  'tr': 224,
  'it': 107,
  'de': 2},
 'ous19_ar': {'fr': 260,
  'en': 980,
  'pt': 139,
  'hi': 81,
  'es': 170,
  'de': 40,
  'tr': 231,
  'it': 99},
 'ous19_fr': {'pt': 229,
  'de': 32,
  'es': 219,
  'ar': 174,
  'en': 969,
  'tr': 76,
  'hi': 58,
  'it': 243},
 'san20_it': {'pt': 111,
  'en': 1181,
  'de': 50,
  'es': 145,
  'fr': 151,
  'tr': 298,
  'hi': 24,
  'ar': 40},
 'gahd24_de': {'en': 1641,
  'fr': 34,
  'es': 25,
  'pt': 47,
  'tr': 177,
  'it': 67,
  'ar': 8,
  'hi': 1},
 'xdomain_tr': {'en': 1359,
  'pt': 109,
  'fr': 52,
  'hi': 58,
  'ar': 40,
  'es': 97,
  'it': 204,
  'de': 81}}

In [25]:
rs1_language = {'bas19_es': {'en': 1162,
  'pt': 195,
  'fr': 125,
  'hi': 42,
  'ar': 98,
  'it': 222,
  'tr': 132,
  'de': 24},
 'for19_pt': {'en': 1246,
  'es': 200,
  'ar': 74,
  'it': 118,
  'fr': 180,
  'hi': 47,
  'de': 25,
  'tr': 110},
 'has21_hi': {'en': 841,
  'pt': 91,
  'es': 298,
  'ar': 281,
  'fr': 145,
  'it': 106,
  'tr': 235,
  'de': 3},
 'ous19_ar': {'en': 1008,
  'es': 163,
  'fr': 228,
  'hi': 81,
  'pt': 132,
  'it': 96,
  'tr': 254,
  'de': 38},
 'ous19_fr': {'es': 215,
  'hi': 70,
  'de': 29,
  'pt': 232,
  'ar': 170,
  'en': 995,
  'it': 227,
  'tr': 62},
 'san20_it': {'en': 1218,
  'de': 43,
  'ar': 42,
  'fr': 161,
  'pt': 119,
  'hi': 31,
  'es': 120,
  'tr': 266},
 'gahd24_de': {'en': 1655,
  'es': 30,
  'pt': 37,
  'fr': 36,
  'tr': 177,
  'it': 50,
  'ar': 10,
  'hi': 5},
 'xdomain_tr': {'en': 1375,
  'es': 90,
  'pt': 121,
  'ar': 59,
  'hi': 39,
  'fr': 77,
  'it': 170,
  'de': 69}}

rs2_language = {'bas19_es': {'en': 1114,
  'fr': 133,
  'hi': 40,
  'pt': 198,
  'it': 253,
  'de': 32,
  'ar': 87,
  'tr': 143},
 'for19_pt': {'en': 1236,
  'fr': 178,
  'ar': 73,
  'es': 206,
  'hi': 58,
  'it': 89,
  'de': 26,
  'tr': 134},
 'has21_hi': {'en': 821,
  'pt': 87,
  'es': 306,
  'fr': 160,
  'ar': 302,
  'tr': 213,
  'it': 109,
  'de': 2},
 'ous19_ar': {'en': 983,
  'hi': 82,
  'fr': 234,
  'pt': 130,
  'es': 163,
  'de': 33,
  'it': 102,
  'tr': 273},
 'ous19_fr': {'en': 1010,
  'de': 24,
  'es': 204,
  'ar': 169,
  'pt': 244,
  'tr': 81,
  'hi': 60,
  'it': 208},
 'san20_it': {'pt': 132,
  'en': 1170,
  'es': 146,
  'fr': 159,
  'ar': 41,
  'tr': 279,
  'hi': 24,
  'de': 49},
 'gahd24_de': {'en': 1711,
  'pt': 27,
  'fr': 22,
  'es': 30,
  'tr': 140,
  'it': 61,
  'ar': 7,
  'hi': 2},
 'xdomain_tr': {'en': 1365,
  'es': 78,
  'it': 205,
  'fr': 50,
  'pt': 126,
  'hi': 65,
  'ar': 44,
  'de': 67}}

rs3_language = {'bas19_es': {'en': 1165,
  'ar': 82,
  'fr': 137,
  'hi': 41,
  'pt': 181,
  'de': 20,
  'it': 229,
  'tr': 145},
 'for19_pt': {'en': 1254,
  'es': 224,
  'it': 107,
  'fr': 155,
  'ar': 82,
  'hi': 55,
  'tr': 93,
  'de': 30},
 'has21_hi': {'en': 817,
  'pt': 96,
  'es': 316,
  'fr': 158,
  'ar': 295,
  'tr': 210,
  'it': 105,
  'de': 3},
 'ous19_ar': {'en': 1026,
  'es': 154,
  'fr': 246,
  'pt': 137,
  'hi': 78,
  'it': 87,
  'de': 34,
  'tr': 238},
 'ous19_fr': {'es': 202,
  'pt': 251,
  'en': 1011,
  'hi': 60,
  'de': 25,
  'ar': 147,
  'it': 233,
  'tr': 71},
 'san20_it': {'en': 1225,
  'pt': 112,
  'de': 51,
  'ar': 34,
  'tr': 272,
  'fr': 145,
  'es': 144,
  'hi': 17},
 'gahd24_de': {'en': 1697,
  'pt': 37,
  'fr': 31,
  'es': 28,
  'it': 57,
  'tr': 142,
  'ar': 6,
  'hi': 2},
 'xdomain_tr': {'fr': 77,
  'pt': 111,
  'en': 1379,
  'hi': 60,
  'de': 69,
  'ar': 58,
  'it': 173,
  'es': 73}}

rs4_language = {'bas19_es': {'pt': 167,
  'fr': 120,
  'en': 1189,
  'hi': 48,
  'it': 224,
  'ar': 87,
  'tr': 141,
  'de': 24},
 'for19_pt': {'en': 1194,
  'hi': 49,
  'es': 212,
  'it': 118,
  'fr': 178,
  'tr': 116,
  'ar': 94,
  'de': 39},
 'has21_hi': {'en': 825,
  'es': 313,
  'ar': 289,
  'fr': 160,
  'pt': 89,
  'tr': 214,
  'it': 107,
  'de': 3},
 'ous19_ar': {'en': 1009,
  'fr': 234,
  'hi': 80,
  'pt': 136,
  'es': 152,
  'de': 33,
  'it': 108,
  'tr': 248},
 'ous19_fr': {'en': 997,
  'es': 218,
  'ar': 149,
  'pt': 242,
  'tr': 79,
  'hi': 64,
  'it': 228,
  'de': 23},
 'san20_it': {'en': 1210,
  'fr': 142,
  'tr': 266,
  'pt': 124,
  'es': 135,
  'de': 43,
  'hi': 32,
  'ar': 48},
 'gahd24_de': {'en': 1628,
  'fr': 45,
  'pt': 44,
  'it': 62,
  'tr': 174,
  'hi': 6,
  'es': 27,
  'ar': 14},
 'xdomain_tr': {'en': 1415,
  'pt': 79,
  'es': 64,
  'fr': 60,
  'ar': 45,
  'it': 213,
  'hi': 43,
  'de': 81}}

rs5_language = {'bas19_es': {'en': 1124,
  'fr': 129,
  'hi': 49,
  'pt': 191,
  'de': 31,
  'it': 233,
  'ar': 84,
  'tr': 159},
 'for19_pt': {'en': 1238,
  'hi': 55,
  'es': 195,
  'ar': 89,
  'fr': 152,
  'tr': 135,
  'de': 34,
  'it': 102},
 'has21_hi': {'en': 822,
  'pt': 94,
  'es': 309,
  'ar': 287,
  'fr': 155,
  'tr': 224,
  'it': 107,
  'de': 2},
 'ous19_ar': {'fr': 260,
  'en': 980,
  'pt': 139,
  'hi': 81,
  'es': 170,
  'de': 40,
  'tr': 231,
  'it': 99},
 'ous19_fr': {'pt': 229,
  'de': 32,
  'es': 219,
  'ar': 174,
  'en': 969,
  'tr': 76,
  'hi': 58,
  'it': 243},
 'san20_it': {'pt': 111,
  'en': 1181,
  'de': 50,
  'es': 145,
  'fr': 151,
  'tr': 298,
  'hi': 24,
  'ar': 40},
 'gahd24_de': {'en': 1641,
  'fr': 34,
  'es': 25,
  'pt': 47,
  'tr': 177,
  'it': 67,
  'ar': 8,
  'hi': 1},
 'xdomain_tr': {'en': 1359,
  'pt': 109,
  'fr': 52,
  'hi': 58,
  'ar': 40,
  'es': 97,
  'it': 204,
  'de': 81}}

In [27]:
from collections import defaultdict
rs_languages = [rs1_language, rs2_language, rs3_language, rs4_language, rs5_language]


# Store both total and count for averaging
value_sum = defaultdict(lambda: defaultdict(int))
value_count = defaultdict(lambda: defaultdict(int))

# Accumulate totals and counts
for rs in rs_languages:
    for target, sources in rs.items():
        for source, value in sources.items():
            value_sum[target][source] += value/2000
            value_count[target][source] += 1

# Compute averages
avg_dict = {
    target: {
        source: round(value_sum[target][source] * 100 / value_count[target][source], 2)
        for source in sources
    }
    for target, sources in value_sum.items()
}

# Print a sample
# for target, sources in list(avg_dict.items()):
#     print(f"{target}: {sources}")
avg_dict

{'bas19_es': {'en': 57.54,
  'pt': 9.32,
  'fr': 6.44,
  'hi': 2.2,
  'ar': 4.38,
  'it': 11.61,
  'tr': 7.2,
  'de': 1.31},
 'for19_pt': {'en': 61.68,
  'es': 10.37,
  'ar': 4.12,
  'it': 5.34,
  'fr': 8.43,
  'hi': 2.64,
  'de': 1.54,
  'tr': 5.88},
 'has21_hi': {'en': 41.26,
  'pt': 4.57,
  'es': 15.42,
  'ar': 14.54,
  'fr': 7.78,
  'it': 5.34,
  'tr': 10.96,
  'de': 0.13},
 'ous19_ar': {'en': 50.06,
  'es': 8.02,
  'fr': 12.02,
  'hi': 4.02,
  'pt': 6.74,
  'it': 4.92,
  'tr': 12.44,
  'de': 1.78},
 'ous19_fr': {'es': 10.58,
  'hi': 3.12,
  'de': 1.33,
  'pt': 11.98,
  'ar': 8.09,
  'en': 49.82,
  'it': 11.39,
  'tr': 3.69},
 'san20_it': {'en': 60.04,
  'de': 2.36,
  'ar': 2.05,
  'fr': 7.58,
  'pt': 5.98,
  'hi': 1.28,
  'es': 6.9,
  'tr': 13.81},
 'gahd24_de': {'en': 83.32,
  'es': 1.4,
  'pt': 1.92,
  'fr': 1.68,
  'tr': 8.1,
  'it': 2.97,
  'ar': 0.45,
  'hi': 0.16},
 'xdomain_tr': {'en': 68.93,
  'es': 4.02,
  'pt': 5.46,
  'ar': 2.46,
  'hi': 2.65,
  'fr': 3.16,
  'it': 9.65

In [155]:
target_to_label

{'bas19_es': {1: 845, 0: 1155},
 'for19_pt': {1: 689, 0: 1311},
 'has21_hi': {1: 843, 0: 1157},
 'ous19_ar': {0: 1238, 1: 762},
 'ous19_fr': {0: 1247, 1: 753},
 'san20_it': {1: 930, 0: 1070},
 'gahd24_de': {0: 1161, 1: 839},
 'xdomain_tr': {0: 1064, 1: 936}}

In [29]:
rs1_label = {'bas19_es': {0: 1168, 1: 832},
 'for19_pt': {1: 691, 0: 1309},
 'has21_hi': {0: 1153, 1: 847},
 'ous19_ar': {0: 1181, 1: 819},
 'ous19_fr': {0: 1224, 1: 776},
 'san20_it': {1: 902, 0: 1098},
 'gahd24_de': {0: 1155, 1: 845},
 'xdomain_tr': {1: 872, 0: 1128}}

rs2_label = {'bas19_es': {1: 832, 0: 1168},
 'for19_pt': {0: 1322, 1: 678},
 'has21_hi': {1: 835, 0: 1165},
 'ous19_ar': {0: 1193, 1: 807},
 'ous19_fr': {0: 1236, 1: 764},
 'san20_it': {1: 970, 0: 1030},
 'gahd24_de': {1: 875, 0: 1125},
 'xdomain_tr': {0: 1124, 1: 876}}

rs3_label = {'bas19_es': {1: 818, 0: 1182},
 'for19_pt': {0: 1310, 1: 690},
 'has21_hi': {1: 832, 0: 1168},
 'ous19_ar': {0: 1186, 1: 814},
 'ous19_fr': {0: 1237, 1: 763},
 'san20_it': {1: 953, 0: 1047},
 'gahd24_de': {0: 1164, 1: 836},
 'xdomain_tr': {0: 1104, 1: 896}}

rs4_label = {'bas19_es': {0: 1145, 1: 855},
 'for19_pt': {1: 671, 0: 1329},
 'has21_hi': {1: 841, 0: 1159},
 'ous19_ar': {0: 1188, 1: 812},
 'ous19_fr': {0: 1223, 1: 777},
 'san20_it': {0: 1058, 1: 942},
 'gahd24_de': {0: 1119, 1: 881},
 'xdomain_tr': {0: 1075, 1: 925}}

rs5_label = {'bas19_es': {1: 845, 0: 1155},
 'for19_pt': {1: 689, 0: 1311},
 'has21_hi': {1: 843, 0: 1157},
 'ous19_ar': {0: 1238, 1: 762},
 'ous19_fr': {0: 1247, 1: 753},
 'san20_it': {1: 930, 0: 1070},
 'gahd24_de': {0: 1161, 1: 839},
 'xdomain_tr': {0: 1064, 1: 936}}

In [30]:
from collections import defaultdict
rs_labels = [rs1_label, rs2_label, rs3_label, rs4_label, rs5_label]


# Store both total and count for averaging
value_sum = defaultdict(lambda: defaultdict(int))
value_count = defaultdict(lambda: defaultdict(int))

# Accumulate totals and counts
for rs in rs_labels:
    for target, sources in rs.items():
        for source, value in sources.items():
            value_sum[target][source] += value/2000
            value_count[target][source] += 1

# Compute averages
avg_dict = {
    target: {
        source: round(value_sum[target][source] * 100 / value_count[target][source], 2)
        for source in sources
    }
    for target, sources in value_sum.items()
}

# Print a sample
# for target, sources in list(avg_dict.items()):
#     print(f"{target}: {sources}")
avg_dict

{'bas19_es': {0: 58.18, 1: 41.82},
 'for19_pt': {1: 34.19, 0: 65.81},
 'has21_hi': {0: 58.02, 1: 41.98},
 'ous19_ar': {0: 59.86, 1: 40.14},
 'ous19_fr': {0: 61.67, 1: 38.33},
 'san20_it': {1: 46.97, 0: 53.03},
 'gahd24_de': {0: 57.24, 1: 42.76},
 'xdomain_tr': {1: 45.05, 0: 54.95}}

In [86]:
from collections import defaultdict

for i, data in enumerate(datas):
    embedd = embeddings[i]
    print(data)
    retrieved = retriever.retrieve_multiple_queries(embedd, num_retrieved=500, exclude_datasets=[data])

    scores = [item['score'] for item in retrieved]

    # Calculate average distances for datasets, languages, and labels
    dataset_scores = defaultdict(list)
    language_scores = defaultdict(list)
    label_scores = defaultdict(list)

    for item, score in zip(retrieved, scores):
        dataset_scores[item['metadata']['dataset_name']].append(score)
        language_scores[item['metadata']['language']].append(score)
        label_scores[item['metadata']['label']].append(score)

    # Compute average distances
    avg_dataset_scores = {k: sum(v) / len(v) for k, v in dataset_scores.items()}
    avg_language_scores = {k: sum(v) / len(v) for k, v in language_scores.items()}
    avg_label_scores = {k: sum(v) / len(v) for k, v in label_scores.items()}

    # Find closest group based on minimum average distance
    closest_dataset = min(avg_dataset_scores, key=avg_dataset_scores.get)
    closest_language = min(avg_language_scores, key=avg_language_scores.get)
    closest_label = min(avg_label_scores, key=avg_label_scores.get)

    # Print results
    print(f"Closest dataset_name (by score): {closest_dataset} (avg distance: {avg_dataset_scores[closest_dataset]:.4f})")
    print(f"Closest language (by score): {closest_language} (avg distance: {avg_language_scores[closest_language]:.4f})")
    print(f"Closest label (by score): {closest_label} (avg distance: {avg_label_scores[closest_label]:.4f})")


bas19_es
Closest dataset_name (by score): has21_hi (avg distance: 0.4574)
Closest language (by score): hi (avg distance: 0.4574)
Closest label (by score): 0 (avg distance: 0.5079)
for19_pt
Closest dataset_name (by score): bas19_es (avg distance: 0.4466)
Closest language (by score): es (avg distance: 0.4466)
Closest label (by score): 0 (avg distance: 0.4637)
has21_hi
Closest dataset_name (by score): ous19_fr (avg distance: 0.5193)
Closest language (by score): fr (avg distance: 0.5193)
Closest label (by score): 0 (avg distance: 0.5801)
ous19_ar
Closest dataset_name (by score): for19_pt (avg distance: 0.5605)
Closest language (by score): pt (avg distance: 0.5605)
Closest label (by score): 0 (avg distance: 0.6369)
ous19_fr
Closest dataset_name (by score): has21_hi (avg distance: 0.4705)
Closest language (by score): hi (avg distance: 0.4705)
Closest label (by score): 0 (avg distance: 0.4944)
san20_it
Closest dataset_name (by score): ous19_fr (avg distance: 0.5304)
Closest language (by score